<a href="https://colab.research.google.com/github/fnastasi/8648_Robotica_Movil_TP_Final/blob/main/Desafio_Exploracion/Desafio_Exploracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafío Exploración

## Implementación
Se implementa con el filtro de partículas Rao-Blackwellizado. En esta versión, solo se utiliza la odometría cruda para actualizar la pose de las partículas. 

Se hicieron pruebas de corrección de odometría utilizando algunas lecturas de lidar pero los resultados obtenidos no fueron aceptables, incluso peores que los que se obtienen solo con odometría cruda.

## Mapeo
Para crear el mapa se cuentan los "hits" y "misses" de cada celda. Se quiso hacer una prueba usando log-odds pero los resultados no fueron aceptables.

No se llegó a hacer un planeamiento de las posiciones que debe recorrer el robot para obtener nueva información del mapa. Simplemente se le indica al robot que se mueva en linea recta hasta que detecte que puede llegar a colisionar, y rotar en el lugar para evitar la colisión.

## Cómo ejecutar la simulación

Se deben cargar los archivos *diffRobot.py*, *imagen_2021_mapa_tp.png* y *ploteo_de_la_simulacion.py*

Luego ejecutar las celdas empezando por la que le sigue a este texto.


In [ ]:
# Debugger
import pdb

# importar archivos google colab
from google.colab import files

In [ ]:
# Librerías que uso
import matplotlib as mlp
%matplotlib notebook

import numpy as np
import matplotlib.image as pimg
# from enum import Enum
import math
import threading
import time
import timeit
import json

from math import *
import numpy.linalg as lng
import argparse
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

import matplotlib.animation as animation
import matplotlib.patches as ppatches

import functools
import sys

import importlib

from scipy import signal

from diffRobot import DiffRobot

# funciones para hacer ploteos
from ploteo_de_la_simulacion import drawRobot
from ploteo_de_la_simulacion import drawRobotPos
from ploteo_de_la_simulacion import drawLidarMeasure
from ploteo_de_la_simulacion import updatedLidarDraw

# función de uso para el inverse sensor model
from skimage.draw import line

# Creo una carpeta para guardar los gráficos
!mkdir plots

In [ ]:
# Clase partícula
class particle():
    def __init__(self,x_mapa,y_mapa,escala,centro_mapa):
        # x_mapa: largo del mapa
        # y_mapa: alto del mapa

        # pose de la partícula
        self.x = 0
        self.y = 0
        self.theta = 0
        
        # peso de la partícula
        self.peso = 1.0

        # mapa de la partícula. Se inicializa tal que no se sabe
        # nada de las grillas. EL MAPA ESTÁ EN LOG ODDS
        self.occ_map = np.zeros((x_mapa,y_mapa))

        # En esta lista se guarda la trayectoria de la partícula
        self.trayectoria = []

        # escala del mapa
        self.occ_map_scale = escala

        # centro del mapa
        self.occ_map_center = centro_mapa

        # hits
        self.hits = np.zeros((x_mapa,y_mapa))

        # misses
        self.misses = np.zeros((x_mapa,y_mapa))

#========================================================================#
    def particle_pos(self):
        # Devuelve la pose de la partícula
        vector = np.array([self.x,self.y,self.theta])

        return vector
#========================================================================#
    def set_particle_pos(self,pose):
        # setter para la pose de la partícula
        self.x = pose[0]
        self.y = pose[1]
        self.theta = pose[2]
#========================================================================#
    def update_log_odds(self,coordenadas_x,coordenadas_y):
        # Recibe dos arreglos "coordenadas_x", "coordenadas_y" que indican qué
        # celdas deben actualizarse, del mapa de grilla. La primer celda, es
        # decir (coordenadas_x[0],coordeandas_y[0]), corresponde a la pose del
        # robot. La última corresponde al lugar donde impactó el lidar. Todas
        # las celdas excepto la última se actualizan indicando que están vacías.
        # La última se actualiza indicando que está ocupada.


        # for i in range(len(coordenadas_x)-1):
        #     x = coordenadas_x[i]
        #     y = coordenadas_y[i]

        #     #p = (-0.9/len(coordenadas_x)) * i + 1

        #     if((0 <= x < self.occ_map.shape[0]) and (0 <= y < self.occ_map.shape[1])):
        #         #(self.occ_map)[x,y] = np.log(p * 0.3/(1-p*0.3)) + (self.occ_map)[x,y]
        #         (self.occ_map)[x,y] = np.log(0.3/(1-0.3)) + (self.occ_map)[x,y]
        
        # x = coordenadas_x[i+1]
        # y = coordenadas_y[i+1]

        # if((0 <= x < self.occ_map.shape[0]) and (0 <= y < self.occ_map.shape[1])):
        #     # Actualizo el punto donde impactó el lidar
        #     #p = (-0.9/len(coordenadas_x)) * (i+1) + 1

        #     #(self.occ_map)[x,y] = np.log(p*0.6/(1-p*0.6)) + (self.occ_map)[x,y]
        #     (self.occ_map)[x,y] = np.log(0.6/(1-0.6)) + (self.occ_map)[x,y]

        for j in range((len(coordenadas_x)) - 1):
            x = coordenadas_x[j]
            y = coordenadas_y[j]
            if((0 <= x < self.occ_map.shape[0]) and (0 <= y < self.occ_map.shape[1])):
                (self.misses)[x,y] = (self.misses)[x,y] + 1

                (self.occ_map)[x,y] = (self.hits)[x,y]/((self.hits)[x,y] + (self.misses)[x,y])


        # print(i)
        x = coordenadas_x[len(coordenadas_x)-1]
        y = coordenadas_y[len(coordenadas_y)-1]

        if((0 <= x < self.occ_map.shape[0]) and (0 <= y < self.occ_map.shape[1])):
            # Actualizo el punto donde impactó el lidar

            (self.hits)[x,y] = (self.hits)[x,y] + 1

            (self.occ_map)[x,y] = (self.hits)[x,y]/((self.hits)[x,y] + (self.misses)[x,y])


#========================================================================#
    def __calculateCellForPosition(self, x, y):
        
        norm_pos = np.array([x,y]) / self.occ_map_scale
        pos_occmap = norm_pos + self.occ_map_center

        if pos_occmap[1] > 0:
            row = int(self.occ_map.shape[0] - pos_occmap[1])
        else:
            row = self.occ_map.shape[0] + 1

        if pos_occmap[0] > 0:
            col = int(pos_occmap[0])
        else:
            col = -1
        return (row, col)
#========================================================================#
    def __amIOutsideOccMap (self, x, y):
        out_of_map = True
        row = -1
        col = -1
        # if self.occ_map_loaded == True:
        (row, col) = self.__calculateCellForPosition(x, y)
        
        if row < 0 or row >= self.occ_map.shape[0]:
            out_of_map = True
        elif col < 0 or col >= self.occ_map.shape[1]:
            out_of_map = True
        else:
            out_of_map = False 
        # else:
            # out_of_map = True
        
        return (out_of_map, row, col)
#========================================================================#
    def __isItFreePosition(self, x, y):
        free_pos = False
        out_of_map = False
        # if self.occ_map_loaded == True:
        (out_of_map, row, col) = self.__amIOutsideOccMap(x,y)
        if  out_of_map == False:
            occ_map_value = self.occ_map[row, col]
            # if (occ_map_value < 0.5):
            if (occ_map_value >= 0.5):
                free_pos = False
            else:
                free_pos = True
        else:
            free_pos = False
        # else:
            # free_pos = True

        return (free_pos, out_of_map)
#========================================================================#
    def ray_casting(self, x, y, theta):
        # Hace raycasting de la pose de la partícula sobre su propio mapa

        # dx = math.cos(theta) *  self.occ_map_scale
        # dy = math.sin(theta) *  self.occ_map_scale
        dx = np.cos(theta) *  self.occ_map_scale
        dy = np.sin(theta) *  self.occ_map_scale
        #dx = np.cos(theta) *  0.
        #dy = np.sin(theta) *  0.1
        free_pos = True
        out_of_map = False

        next_x = x
        next_y = y
        last_free_pos_x = x
        last_free_pos_y = y
        while free_pos == True and out_of_map == False:
            last_free_pos_x = next_x
            last_free_pos_y = next_y
            next_x += dx
            next_y += dy
            (free_pos, out_of_map) = self.__isItFreePosition(next_x, next_y)
            #if free_pos == False or out_of_map == True:
            #    next_x = last_free_pos_x
            #    next_y = last_free_pos_y
            #    free_pos = True
            #    out_of_map = False
            #    ddx = dx / 8
            #    ddy = dy / 8
            #    while free_pos == True and out_of_map == False:
            #       last_free_pos_x = next_x
            #        last_free_pos_y = next_y
            #        next_x += ddx
            #        next_y += ddy
            #        (free_pos, out_of_map) = self.__isItFreePosition(next_x, next_y)
            #else:
            #    pass
        
        distance_measured = np.sqrt((last_free_pos_x - x)**2 + (last_free_pos_y - y)**2)
        #error = np.random.normal(scale = 0.015 * distance_measured)
        # error = np.random.normal(scale = 0.15 * distance_measured)
        return  distance_measured
#========================================================================#
    def update_particle_motion(self,v_0,w_0,t,alfa):
        # Se actualiza la pose de la partícula según la odometría. Antes de hacerlo
        # Se guarda la pose actual en la trayectoria

        (self.trayectoria).append(self.particle_pos())

        v = v_0 + np.random.normal(0,alfa[0]*abs(v_0) + alfa[1]*abs(w_0))
        w = w_0 + np.random.normal(0,alfa[2]*abs(v_0) + alfa[3]*abs(w_0))

        theta_new = self.theta + w * t
        x_new = self.x + v * cos(theta_new) * t
        y_new = self.y + v * sin(theta_new) * t

        while(theta_new > np.pi):
          theta_new -= 2 * np.pi
        while(theta_new < -np.pi):
          theta_new += 2 * np.pi

        self.set_particle_pos(np.array([x_new,y_new,theta_new]))

In [ ]:
# Clase filtro de partículas

class particle_filter():
    def __init__(self,N,K,x_mapa,y_mapa,escala,centro_mapa,alfa,l,p):
        
        # inicialización del filtro de partículas. Se crean "N" partículas
        self.noise_param = alfa
        
        self.lidar_resolution = l
        self.lidar_pos = p
        
        self.particles = []
        self.pesos = []
        
        for i in range(0,N):
            # creo una partícula
            p = particle(x_mapa,y_mapa,escala,centro_mapa)

            self.particles.append(p)
            self.pesos.append(1.0)
        
        # Número de partículas
        self.N = N

        # Cantidad de mediciones que se usan para odometría, antes de usar la 
        # medición para mapear y para calcular los pesos
        self.K = K

        # Contador interno para saber cuándo actualizar los mapas y cuando hacer
        # odometría
        self.k_counter = 0

        # alto del mapa
        self.y_mapa = y_mapa

        # ancho del mapa
        self.x_mapa = x_mapa

        # escala del mapa
        self.map_scale = escala

        # centro del mapa
        self.map_center = centro_mapa
#========================================================================#
    def best_particle_pos(self):
        # Devuelve la posición de la partícula con el mejor peso

        peso_max = self.particles[0].peso
        best_particle = 0

        for i in range(1,self.N):
            peso_new = self.particles[i].peso
            if(peso_new > peso_max):
                peso_max = peso_new
                best_particle = i
        

        return best_particle
#========================================================================#
    #def medicion_esperada(self,x,y,theta,paso):
    def medicion_esperada(self,n_particula,paso):

        dtheta = 2 * np.pi / self.lidar_resolution
        dtheta = paso * dtheta
        lidar_angle = 0
        measure_values = np.zeros(int(self.lidar_resolution / paso))
        x = (self.particles[n_particula]).x
        y = (self.particles[n_particula]).y
        theta = (self.particles[n_particula]).theta
        for i in range(0, int(self.lidar_resolution / paso)):
            measure_values[i] = (self.particles[n_particula]).ray_casting(x, y, theta - self.lidar_pos[2] + lidar_angle)
            lidar_angle += dtheta
        return measure_values
#========================================================================#
    def crear_nube_puntos(self,vector):
        
        N = len(vector)
        nube = np.zeros((2,N))
        
        dtheta = 2 * np.pi / N
        lidar_angle = 0
        
        for i in range(N):
            x_i = vector[i] * np.cos(lidar_angle - self.lidar_pos[2])
            y_i = vector[i] * np.sin(lidar_angle - self.lidar_pos[2])
            nube[0,i] = x_i
            nube[1,i] = y_i
            lidar_angle += dtheta
        return nube
#========================================================================#
    def closest_point_matching(self,X, P):
      
      P_matched = P.copy()
      
      # cantidad de puntos
      N = X.shape[1]
      orden = np.random.permutation(N)
      for l in range(0,N):
          # elijo un punto de la nube de puntos
          i = orden[l]
          x_i = X[:,i]
          # busco el punto más cercano
          dist_min = np.linalg.norm(x_i - P_matched[:,i])
          k = i
          for m in range(l+1,N):
              j = orden[m]
              dist = np.linalg.norm(x_i - P_matched[:,j])
              if(dist<dist_min):
                  x_aux = X[:,j]
                  p_aux = P_matched[:,i]
                  delta_E = dist - dist_min - np.linalg.norm(x_aux - P_matched[:,j]) + np.linalg.norm(x_aux - p_aux)
                  if(delta_E < 0):
                      dist_min = dist
                      k = j
          aux = P_matched[:,k].copy()
          P_matched[:,k] = P_matched[:,i].copy()
          P_matched[:,i] = aux
              
      return P_matched
#========================================================================#
    def matchear_mediciones_odometria(self,nube_h_original,nube_measurements,N,n_particle):

        e_old = 1000
        for i in range(3):
            
            #calculate RMSE
            e = 0
            for j in range(0,nube_h_original.shape[1]):
                
                e = e+(nube_h_original[0,j]-nube_measurements[0,j])**2 + (nube_h_original[1,j]-nube_measurements[1,j])**2
            
            e = np.sqrt(e/nube_h_original.shape[1])

            if(abs(e - e_old) < 1e-2):
                break
            
            e_old = e
            # pdb.set_trace()
            nube_h_original = self.closest_point_matching(nube_measurements,nube_h_original)
            
            #substract center of mass
            mx = np.transpose([np.mean(nube_measurements,1)])
            mp = np.transpose([np.mean(nube_h_original,1)])
            X_prime = nube_measurements-mx
            P_prime = nube_h_original-mp
            
            #singular value decomposition
            W = np.dot(X_prime,np.transpose(P_prime))
            U, s, V = np.linalg.svd(W)

            #calculate rotation and translation
            R = np.dot(U,np.transpose(V))
            t = mx-np.dot(R,mp)
        
            #apply transformation
            nube_h_original = np.dot(R,nube_h_original)+t
            # pdb.set_trace()
            # le aplico la transformación a la partícula
            pose_particula = self.particles[n_particle].particle_pos()
            #pose_particula[0] = pose_particula[0] - t[0][0]
            #pose_particula[1] = pose_particula[1] - t[1][0]
            #aux = R @ np.array([[1],[0]])
            #alfa = np.arctan2(aux[1][0],aux[0][0])
            #pose_particula[2] = pose_particula[2] - alfa

            R_aux = np.array([[np.cos(pose_particula[2]),-np.sin(pose_particula[2])],[np.sin(pose_particula[2]),np.cos(pose_particula[2])]])
            t_aux = np.array([[pose_particula[0]],[pose_particula[1]]])

            t_aux = R_aux @ (-np.linalg.inv(R) @ t) + t_aux
            pose_particula[0] = t_aux[0][0]
            pose_particula[1] = t_aux[1][0]
            pose_particula[2] = pose_particula[2]


            # guardo la pose en la partícula
            self.particles[n_particle].set_particle_pos(pose_particula)

        # return nube_h_original
#========================================================================#
    def actualizar_poses_odometria_cruda(self,v,w,T):
        # Se actualiza la pose de las partículas según la odometría cruda y las 
        # pdb.set_trace()
        # paso = 1
        # measurements = measurements[0::paso]
        # nube_measurements_aux = self.crear_nube_puntos(measurements)
        
        # M = len(measurements)
        # pdb.set_trace()
        for i in range(self.N):
            # Actualizo la partícula con la odometría cruda
            
            (self.particles[i]).update_particle_motion(v,w,T,self.noise_param)
            
#========================================================================#
    def matchear_mediciones_pesos(self,nube_h_original,nube_measurements):

        e_old = 1000
        for i in range(10):
            
            #calculate RMSE
            e = 0
            for j in range(0,nube_h_original.shape[1]):
                
              e = e+(nube_h_original[0,j]-nube_measurements[0,j])**2 + (nube_h_original[1,j]-nube_measurements[1,j])**2
            
            e = np.sqrt(e/nube_h_original.shape[1])

            if(abs(e - e_old) < 1e-2):
                break
            
            e_old = e
             
            nube_h_original = self.closest_point_matching(nube_measurements,nube_h_original)

            #substract center of mass
            mx = np.transpose([np.mean(nube_measurements,1)])
            mp = np.transpose([np.mean(nube_h_original,1)])
            X_prime = nube_measurements-mx
            P_prime = nube_h_original-mp
            
            #singular value decomposition
            W = np.dot(X_prime,np.transpose(P_prime))
            U, s, V = np.linalg.svd(W)
    
            #calculate rotation and translation
            R = np.dot(U,np.transpose(V))
            t = mx-np.dot(R,mp)
        
            #apply transformation
            nube_h_original = np.dot(R,nube_h_original)+t

        return nube_h_original
#========================================================================#
    def actualizar_mapas(self,measurements):
        # Se actualizan los mapas de todas las partículas usando las lecturas del
        # lidar.

        paso = 1
        measurements = measurements[0::paso]
        nube_measurements = self.crear_nube_puntos(measurements)

        centro_mapa_aux = np.copy(self.map_center)
        centro_mapa_aux.shape = (2,1)

        for i in range(self.N):
            # Se transforma la nube de puntos relativa a la pose de la partícula a 
            # una donde los puntos son coordenadas en el mapa
            
            pose_particula = (self.particles[i]).particle_pos()
            theta_p = pose_particula[2]

            R = np.array([[np.cos(theta_p),np.sin(-theta_p)],[np.sin(theta_p),np.cos(theta_p)]])

            t = pose_particula[0:2]
            t.shape = (2,1)

            # Este cálculo da como resultado una nube de puntos con coordenadas
            # respecto del sistema inercial
            nube_measurements_real = R @ nube_measurements + t

            # medicion_esperada = self.medicion_esperada(i,paso)
            # nube_medicion_esperada = self.crear_nube_puntos(medicion_esperada)
            # nube_medicion_esperada = R @ nube_medicion_esperada + t

            # # la matcheo contra lo que ya tenía (?)
            # nube_measurements_real = self.matchear_mediciones_pesos(nube_measurements_real,nube_medicion_esperada)

            # Convierto esos puntos a coordenadas de las celdas del mapa
            nube_measurements_grilla = np.floor(nube_measurements_real / self.map_scale + centro_mapa_aux)

            # Lo mismo para la pose de la partícula

            pose_particula_grilla = (np.floor(t / self.map_scale + centro_mapa_aux))
            # pdb.set_trace()
            # Para cada uno de esos puntos me fijo cuáles son las celdas del mapa
            # por las que pasa el haz del lidar antes de impactar
            for j in range(len(measurements)):
                # rr: filas
                # cc: columnas
                rr, cc = line(int(pose_particula_grilla[0,0]), int(pose_particula_grilla[1,0]), int(nube_measurements_grilla[0,j]), int(nube_measurements_grilla[1,j]))

                # Se actualiza el log odds de las celdas que devuelve la función "line"
                (self.particles[i]).update_log_odds(rr,cc)
#========================================================================#
    def resample_particles(self,w):
        
        print(w)
        
        neff = 0
        # pdb.set_trace()
        eta = np.sum(w)
        
        # normalizo los pesos
        w = np.divide(w,eta)
        


        # calculo el número efectivo de partículas
        neff = 1/np.sum(np.square(w))

        L = len(w)

        #if(neff >= int(self.N/2)):
        if(neff>=L/2):

            pass

        else:
            print(w)
            c = []    
            c.append(w[0])
            
            for i in range(1,self.N):
                c.append(c[i-1] + w[i])
            
            step = 1/(self.N)
            
            seed = np.random.uniform(0,step)
            
            i = 0
            u = seed
            p_sampled = []

            for h in range(self.N):

                while u > c[i]:
                    i = i + 1
                particula_elegida = self.particles[i]

                particula_elegida.peso = w[i]
                p_sampled.append(particula_elegida)
                u = u + 1/self.N
            
            self.particles = p_sampled
 #========================================================================#
    def resamplear(self,measurements):
        
        paso = 1
        
        # vector de pesos weights
        weights = []
        measurements = measurements[0::paso]
        nube_measurements = self.crear_nube_puntos(measurements)
        M = len(measurements)
        for i in range(self.N):
            
            # busco el vector de medición esperada
            medicion_esperada = self.medicion_esperada(i,paso)
            
            # matcheo
            # creo las nubes de puntos a partir de las mediciones
            nube_h_original = self.crear_nube_puntos(medicion_esperada)


            # a partir de ambas nubes de puntos, hago el matcheo de ambas
            # nube_h_original = self.matchear_mediciones_pesos(nube_h_original,nube_measurements,M)
            
            # a partir de la nube de puntos, calculo las distancias
            # h = np.zeros(M)
            # for j in range(M):
            #    h[j] = np.linalg.norm(nube_h_original[:,j])
            #h = medicion_esperada
            
            # diff = h - measurements
            
            # variance = (0.15 * measurements)**2
            
            # prob = np.prod(np.exp(-(diff)**2 / (2 * variance)) / (np.sqrt(variance * 2 * np.pi)))
            #prob = np.sum(-0.5 * np.log(2 * np.pi * variance) + (-(diff)**2) / (2 * variance))
            #print(prob)
            prob = 0
            # pdb.set_trace()

            theta_p = (self.particles[i].particle_pos())[2]
            R_p = np.array([[np.cos(theta_p),np.sin(-theta_p)],[np.sin(theta_p),np.cos(theta_p)]])

            for j in range(M):
                prob += np.linalg.norm(R_p @ (nube_h_original[:,j] - nube_measurements[:,j]))
            # pdb.set_trace()
            prob = 1/prob
            weights.append(prob)
            (self.particles)[i].peso = prob
          
         
        self.resample_particles(weights)
#========================================================================#
    def ICP_transform(self,nube_esperada,nube_measurements):
        # Devuelve la transformación entre 2 nubes de puntos

        T = np.eye(3)

        e_old = 1000
        for i in range(100):
            
            #calculate RMSE
            e = 0
            for j in range(0,nube_esperada.shape[1]):
                
                e = e+(nube_esperada[0,j]-nube_measurements[0,j])**2 + (nube_esperada[1,j]-nube_measurements[1,j])**2
            
            e = np.sqrt(e/nube_esperada.shape[1])

            if(abs(e - e_old) < 1e-3):
                break
            
            e_old = e
            
            nube_measurements = self.closest_point_matching(nube_esperada,nube_measurements)
            
            #substract center of mass
            mx = np.transpose([np.mean(nube_esperada,1)])
            mp = np.transpose([np.mean(nube_measurements,1)])
            X_prime = nube_esperada-mx
            P_prime = nube_measurements-mp
            
            #singular value decomposition
            W = np.dot(X_prime,np.transpose(P_prime))
            U, s, V = np.linalg.svd(W)

            #calculate rotation and translation
            R = np.dot(U,np.transpose(V))
            t = mx-np.dot(R,mp)
        
            # apply transformation
            nube_measurements = np.dot(R,nube_measurements)+t

            # Acumulo las transformaciones para después retornarla
            T_aux = np.array([[R[0][0],R[0][1],t[0][0]],[R[1][0],R[1][1],t[1][0]],[0,0,1]])

            T = T @ T_aux

        return (T,e)

# #========================================================================#
#     def matchear_odometria(self,measurements):
        
#         # Creo la nube de puntos centrada en la partícula
#         paso = 1
#         measurements = measurements[0::paso]
#         nube_measurements_original = self.crear_nube_puntos(measurements)
        
#         M = len(measurements)
#         for i in range(self.N):
#             # busco el vector de medición esperada
#             # pdb.set_trace()
#             medicion_esperada = self.medicion_esperada(i,paso)
            
#             # creo las nubes de puntos
#             nube_esperada = self.crear_nube_puntos(medicion_esperada)
#             nube_measurements = np.copy(nube_measurements_original)
#             # Hago el matcheo de la nube medida respecto de la nube esperada
#             (T,e) = self.ICP_transform(nube_esperada,nube_measurements)
#             # pdb.set_trace()
#             print(e)
#             if(e<1):
                

#                 # Extraigo la matriz R y el vector t
#                 R = T[0:2,0:2]
#                 t = T[0:2,2:]

#                 # De la matriz R extraigo el ángulo de rotación
#                 v = R @ np.array([[1],[0]])

#                 delta_theta = np.arctan2(v[1][0],v[0][0])

#                 # Aplico la transformación a la partícula
#                 pose_particula = self.particles[i].particle_pos()

#                 x = pose_particula[0]
#                 y = pose_particula[1]
#                 theta = pose_particula[2]

#                 xp = np.array([[x],[y]])
#                 Rp = np.array([[np.cos(theta),np.sin(-theta)],[np.sin(theta),np.cos(theta)]])

#                 p = Rp @ t + xp
#                 theta_new = theta + delta_theta

#                 while(theta_new > np.pi):
#                     theta_new -= 2 * np.pi
#                 while(theta_new < -np.pi):
#                     theta_new += 2 * np.pi

#                 pose_nueva = np.array([p[0][0],p[1][0],theta])
#                 self.particles[i].set_particle_pos(pose_nueva)
#========================================================================#
    def matchear_odometria(self,measurements_old,measurements_new):
        
        nube_measurements_old = self.crear_nube_puntos(measurements_old)
        nube_measurements_new = self.crear_nube_puntos(measurements_new)
        
        # Hago el matcheo de la nube medida respecto de la nube esperada
        (T,e) = self.ICP_transform(nube_measurements_old,nube_measurements_new)
        # (T,e) = self.ICP_transform(nube_measurements_new,nube_measurements_old)
        # pdb.set_trace()
        if(e<1):
            # Extraigo la matriz R y el vector t
            R = T[0:2,0:2]
            t = T[0:2,2:]

            # De la matriz R extraigo el ángulo de rotación
            v = R @ np.array([[1],[0]])

            delta_theta = np.arctan2(v[1][0],v[0][0])
            for i in range(self.N):
                # Aplico la transformación a la partícula
                # pose_particula = self.particles[i].particle_pos()
                l = len(self.particles[i].trayectoria)
                pose_particula = self.particles[i].trayectoria[l-1]

                x = pose_particula[0]
                y = pose_particula[1]
                theta = pose_particula[2]

                xp = np.array([[x],[y]])
                Rp = np.array([[np.cos(theta),np.sin(-theta)],[np.sin(theta),np.cos(theta)]])

                p = Rp @ t + xp
                theta_new = theta + delta_theta

                while(theta_new > np.pi):
                    theta_new -= 2 * np.pi
                while(theta_new < -np.pi):
                    theta_new += 2 * np.pi

                pose_nueva = np.array([p[0][0],p[1][0],theta_new])
                pose_actual = self.particles[i].particle_pos()
                pose_nueva = (0.4 * pose_nueva + 0.6 * pose_actual)
                self.particles[i].set_particle_pos(pose_nueva)

In [ ]:
# Utils: funciones que se usan en la simulación

def motion_model_velocity(od_old,od_new,dt):
    # Recibe 2 lecturas de odometría y calcula la velocidad lineal y anugular
    
    v = np.sqrt((od_old[0] - od_new[0])**2 + (od_old[1] - od_new[1])**2) / dt

    delta_theta = od_new[2] - od_old[2]

    while(delta_theta < -0.5 * dt):
        delta_theta += 2*np.pi
    while(delta_theta > 0.5 * dt):
        delta_theta -= 2*np.pi
    
    w = (delta_theta)/dt

    return (v,w)
#========================================================================#
def plotear_mapa(ax,mapa):
    # Plotea el mapa de las partículas en el ax

    # Convierto el mapa de log-odds a probabilidades
    # mapa_aux = 1/(1 + np.exp(mapa))
    mapa_aux = mapa
    # for i in range(mapa_aux.shape[0]):
    #     for j in range(mapa_aux.shape[1]):
    #         if(mapa_aux[i,j] > 0.5):
    #             mapa_aux[i,j] = 1
    #         elif(mapa_aux[i,j] < 0.5):
    #             mapa_aux[i,j] = 0
    #         else:
    #             mapa_aux[i,j] = 0.5    

    ax.clear()

    mapa_mejor_particula = ax.pcolormesh(np.transpose(1-mapa_aux))
    ax.axis('equal')

    return mapa_mejor_particula
#========================================================================#
def actualizar_mapa(ax,mapa):
    ax.clear()
    # Plotea el mapa de las partículas en el ax

    # Convierto el mapa de log-odds a probabilidades
    # mapa_aux = 1 - 1/(1 + np.exp(mapa))
    mapa_aux = mapa
    mapa_mejor_particula = ax.pcolormesh(np.transpose(1-mapa_aux))
    ax.axis('equal')
    return mapa_mejor_particula

#========================================================================#
def plotear_trayectoria(ax,trayectoria):
    # Se plotea la trayectoria de la mejor partícula en el ax

    points_x = np.zeros(len(trayectoria))
    points_y = np.zeros(len(trayectoria))

    for i in range(0, len(trayectoria)):

        points_x[i] = trayectoria[i,0]
        points_y[i] = trayectoria[i,1]

    trayectoria_mejor_particula = ax.scatter(points_x, points_y, c = 'g',marker = '.')

    return trayectoria_mejor_particula
#========================================================================#
# def actualizar_trayectoria(trayectoria_mejor_particula, trayectoria, map_center, map_scale):
def actualizar_trayectoria(ax, trayectoria, map_center, map_scale):
    # Actualiza la trayectoria en el plot

    points_x = np.zeros(len(trayectoria))
    points_y = np.zeros(len(trayectoria))
    # pdb.set_trace()
    for i in range(0, len(trayectoria)):
        points_x[i] = (trayectoria[i][0])*map_scale[0] + map_center[0]
        points_y[i] = (trayectoria[i][1])*map_scale[1] + map_center[1]

    # trayectoria_mejor_particula.set_offsets(np.c_[points_x, points_y])
    trayectoria_mejor_particula = ax.scatter(points_x, points_y, c = 'g',marker = '.')
    return trayectoria_mejor_particula
#========================================================================#
# Detección de posible colisión durante la localización, es decir, no se
# sabe todavía donde está el robot.

def anti_colision(medidas_colision):
    # se asume que se está yendo hacia adelante

    dist_seguridad = 0.05  #metros
    radio_robot = 0.175 # metros

    prob_colision = False
    N = len(medidas_colision)  # va a valer 144
    puntos_a_tomar = int(N / 4)

    for i in range(puntos_a_tomar):
        aux = medidas_colision[int(N/2) - int(puntos_a_tomar/2) + i]

        if(aux <= radio_robot + dist_seguridad):
            prob_colision = True

    return prob_colision
#========================================================================#
# def angulo_salida_anti_colision(medidas_colision):

#     angulos = np.linspace(-np.pi,np.pi,144)
#     dist_acum = 0
#     acum = 0
#     N = len(medidas_colision)  # va a valer 144
#     #puntos_a_tomar = int(N / 2)
#     #dist_max = medidas_colision[0]

#     for j in range(N):
#         acum += angulos[j]*medidas_colision[j]
#         dist_acum += medidas_colision[j]

#     # Tengo que rotar este ángulo
#     alfa = acum/dist_acum
#     # max = medidas_colision[0]
#     # max_pos = 0
#     # for j in range(1,N):
#     #     if(medidas_colision[j] > max):
#     #         max = medidas_colision[j]
#     #         max_pos = j
    
#     # alfa = angulos[j]


#     return alfa
#========================================================================#
def angulo_salida_anti_colision(medidas_colision):

    # angulos = np.linspace(-np.pi,np.pi,144)
    # dist_acum = 0
    # acum = 0
    # N = len(medidas_colision)  # va a valer 144
    # puntos_a_tomar = int(N / 2)
    # # dist_max = medidas_colision[0]

    # # for j in range(N):
    # #     acum += angulos[j]*medidas_colision[j]
    # #     dist_acum += medidas_colision[j]

    # # # Tengo que rotar este ángulo
    # # alfa = acum/dist_acum
    # for j in range(puntos_a_tomar):
    #     acum += (-np.pi + angulos[j+int(N/2) - int(puntos_a_tomar/2)])*medidas_colision[j + int(N/2) - int(puntos_a_tomar/2)]
    #     dist_acum += medidas_colision[j + int(N/2) - int(puntos_a_tomar/2)]

    # # Tengo que rotar este ángulo
    # alfa = acum/dist_acum

    alfa = np.pi / 8

    return alfa
#========================================================================#  
# Dibuja las partículas en el plot  
def drawparticles(ax, p, map_center, map_scale):
    points_x = np.zeros(len(p))
    points_y = np.zeros(len(p))

    for i in range(0, len(p)):
        points_x[i] = (p[i].x)*map_scale[0] + map_center[0]
        points_y[i] = (p[i].y)*map_scale[1] + map_center[1]

    particlepoints = ax.scatter(points_x, points_y, c = 'g',marker = '.')

    return particlepoints

In [ ]:
# Desafío Exploración
################################################################################
# cierro los plots
plt.close('all')


# Pose inicial REAL del robot (ground truth)
x0 = 0
y0 = 0
theta0 = 0

# Creo una instancia de DiffRobot
my_robot = DiffRobot(x0,y0,theta0)

# Cargo el mapa de grilla de ocupación
# Como resolución (metros/píxeles) uso el valor 5 metros/126 píxeles

occupation_map_file = "imagen_2021_mapa_tp.png"
occupation_map_scale_factor = 5/126

my_robot.loadOccupationMap(occupation_map_file, occupation_map_scale_factor)

# se guardan estas variables para hacer los gráficos del mapa y de la ubicación del robot
(occ_map_height, occ_map_width) = my_robot.getOccMapSizeInWorld()
occ_map = my_robot.getOccMap()

# pdb.set_trace()

########################################################################################
# Se crea el gráfico donde se va a plottear la simulación
fig = plt.figure()
#ax = fig.add_subplot(1,1,1)
ax = fig.add_subplot(1,2,1)
ax.pcolormesh(np.flipud(occ_map))
ax.axis('off')
status_text = ax.text(0, -5, "")


# gráfico para la mejor partícula 
ax2 = fig.add_subplot(1,2,2)
ax2.axis('off')
ax2.text(0, -5, "Mapa de la mejor partícula")
ax2.axis('equal')
map_center = (occ_map.shape[0] /2.0, occ_map.shape[1] / 2.0)
map_scale = (occ_map.shape[0] / occ_map_width, occ_map.shape[1] / occ_map_height)

plt.ion()
(gt_icon, gt_bearing) = drawRobot(ax, 0 + map_center[0], 0 + map_center[1], 0, my_robot.diameter * map_scale[0], 'g')
(od_icon, od_bearing) = drawRobot(ax, 0 + map_center[0], 0 + map_center[1], 0, my_robot.diameter * map_scale[0], 'grey')



plt.draw()
plt.pause(1)

lidar_values = np.zeros(my_robot.getLidarResolution())
lidarPoints = drawLidarMeasure(ax, [0 ,0 ,0], my_robot.getLidarResolution(), lidar_values)

# Cantidad de iteraciones de la simulación
simulation_steps = 1000

# Flags varios #################################################################

# Flags de colisión-----
anti_colision_mode = True
posible_colision = False
rutina_colision = False

# Parámetros de la localización ################################################

# Parámetros del filtro de partículas-----
error_parameters = np.array([0.10, 0.20, 0.10, 0.20, 0.25, 0.25])

res_lidar = my_robot.getLidarResolution() # Resolución del lidar

pos_lidar = np.array([0.09, 0.0, math.pi]) # Pose LIDAR respecto al robot

N = 5 # Número de partículas

K = 3 # Número de mediciones antes de hacer una actualización del mapa y de los
# pesos

x_mapa = occ_map.shape[0] # Ancho del mapa
y_mapa = occ_map.shape[1] # Alto del mapa

# Se crea el filtro de partículas
centro_del_mapa = np.copy(map_center)
#centro_del_mapa.shape = (2,1)
filtro_de_particulas = particle_filter(N,K,x_mapa,y_mapa,occupation_map_scale_factor,centro_del_mapa,error_parameters,res_lidar,pos_lidar)


# instantes de tiempo de la simulación lectura de odometría del
# robot. Se usarán para calcular después las predicciones y 
# correcciones.
t_old = my_robot.getSimulationTime()

# lectura de odometría
od_old = my_robot.getOdometry()

# lectura de Lidar
# while(1):
#     mediciones_old = my_robot.getLidarMeaurement()
#     if(mediciones_old.all() != 0):
#         break

# Se plotea el mapa de la mejor partícula por primera vez y luego se actualizará
mapa_mejor_particula = plotear_mapa(ax2,(filtro_de_particulas.particles[filtro_de_particulas.best_particle_pos()]).occ_map)

# Se plotea la trayectoria de la mejor partícula por primera vez y luego se la actualizará
# trayectoria_mejor_particula = plotear_trayectoria(ax2,(filtro_de_particulas.particles[filtro_de_particulas.best_particle_pos()]).trayectoria)
#############################################################################################
# Se configura al robot y comienza la simulacón
my_robot.setLinearVelocity(0.0)
my_robot.setAngularVelocity(0.0)
my_robot.enableBumper()
my_robot.turnLidar_ON()

my_robot.startSimulation(print_status=False)

# # Antes de comenzar, se hace una primer mapeo de las partículas
# while(1):
#     mediciones = my_robot.getLidarMeaurement()
#     if(mediciones.all() != 0):
#         break

# filtro_de_particulas.actualizar_mapas(mediciones)

v_loc = 0.02
w_loc = 0.0

# Comienza el ciclo ############################################################
mediciones_old = my_robot.getLidarMeaurement()
for i in range(simulation_steps):
# 1: FAST SLAM #################################################################
    # Nuevas lecturas de odometría y de tiempo
    t_new = my_robot.getSimulationTime()
    T = t_new - t_old
    od_new = my_robot.getOdometry()

    # Se toma una medición de Lidar
    # mediciones_new = my_robot.getLidarMeaurement()

    if(T!=0):
        (v,w) = motion_model_velocity(od_old,od_new,T)
        # print(v,w)
        t_old = t_new
        od_old = od_new

        # Se le pasan estas mediciones al filtro de partículas para que recalcule
        # la pose de las partículas, actualice los mapas de las partículas y 
        # recalcule los pesos, según corresponda.

        # Hago scan matching de las mediciones con el mapa de cada partícula y
        # luego actualizo según odometría cruda
        filtro_de_particulas.actualizar_poses_odometria_cruda(v,w,T)
        # filtro_de_particulas.FAST_SLAM(v,w,T,mediciones)
        if(filtro_de_particulas.k_counter < filtro_de_particulas.K):

            # Luego, se toma una medición de Lidar para hacer scan matching y 
            # corregir las poses
            # mediciones = my_robot.getLidarMeaurement()
            # if(mediciones.all() != 0):
            #     filtro_de_particulas.matchear_odometria(mediciones)
            # mediciones_new = my_robot.getLidarMeaurement()
            # if(mediciones_new.all() != 0 and mediciones_new.all() != 0):
            #     filtro_de_particulas.matchear_odometria(mediciones_old,mediciones_new)
            #     mediciones_old = mediciones_new
                
            # Incremento el contador en 1
            filtro_de_particulas.k_counter = filtro_de_particulas.k_counter + 1
        else:
            print("actualizo mapas")

            # Actualización de mapas y de pesos.
            mediciones = my_robot.getLidarMeaurement()
            filtro_de_particulas.actualizar_mapas(mediciones)

            # Si es necesario se resamplea
            mediciones = my_robot.getLidarMeaurement()
            filtro_de_particulas.resamplear(mediciones)

            # Se reinicia el contador
            filtro_de_particulas.k_counter = 0
        
# 2: Seteo de velocidades ######################################################

    # Me fijo si hay una posibilidad de chocar con algo
    if(rutina_colision == False):
        medidas_colision = my_robot.getLidarMeaurement()

        if(medidas_colision.all() != 0):
            posible_colision = anti_colision(medidas_colision)
        else:
            posible_colision = False
        
        if(posible_colision == True):
            # Aviso que estoy en rutina de colisión
            rutina_colision = True

            # Detengo al robot para evitar la colisión
            my_robot.setLinearVelocity(0.0)
            my_robot.setAngularVelocity(0.0)

            # Calculo un nuevo ángulo de rotación, para "escapar" de la 
            # colisión
            alfa = angulo_salida_anti_colision(medidas_colision)
            # alfa = - np.pi/4
            # Seteo la velocidad angular

            if(alfa > 0):
                my_robot.setAngularVelocity(0.1)
            else:
                my_robot.setAngularVelocity(-0.1)

            # Me guardo el ángulo de rotación actual, para después chequear
            # si ya roté
            theta_old = my_robot.getOdometry()[2]

        else:
            # Si no hay posibilidades de colisión, se setean las velocidades 
            # del robot según lo que se requiera

            # Se calculan las velocidades
            # TODO: Agregar función de cálculo de velocidades


            my_robot.setLinearVelocity(v_loc)
            my_robot.setAngularVelocity(w_loc)

    # Si ya venía con la rutina de colisión, sigo rotando
    else:
        # Me fijo si ya roté
        if(abs(my_robot.getOdometry()[2] - theta_old) >= abs(alfa)):
            # Si roté, aviso que salgo de la rutina de colisión y detengo
            # al robot
            rutina_colision = False
            my_robot.setLinearVelocity(0.0)
            my_robot.setAngularVelocity(0.0)

# 3: Ploteo de la simulación ==================================================# 
    # recupero la mejor partícula
    best_particle = filtro_de_particulas.best_particle_pos()

    # ploteo el mapa de la mejor partícula
    mapa_mejor_particula = plotear_mapa(ax2,(filtro_de_particulas.particles[best_particle]).occ_map)

    # ploteo las partículas
    drawparticles(ax2, filtro_de_particulas.particles, map_center, map_scale)

    # ploteo la trayectoria de la mejor partícula
    # actualizar_trayectoria(trayectoria_mejor_particula, (filtro_de_particulas.particles[best_particle]).trayectoria, map_center, map_scale)
    # actualizar_trayectoria(ax2, (filtro_de_particulas.particles[best_particle]).trayectoria, map_center, map_scale)
    
    print("GT: ", my_robot.getGroundTruth(), "PF: ", filtro_de_particulas.particles[best_particle].particle_pos(),"t: ",my_robot.getSimulationTime())

    # The state of the robot is consulted for drawing purpose
    # To update de draw of robot's status could take more time that
    # simulation step (dt = 0.1 seg.)
    # As simulation run in an independent thread the draw will be refreshing 
    # at a lower frequency.
    od = my_robot.getOdometry()
    gt = my_robot.getGroundTruth()

    gt_icon.center = gt[0] * map_scale[0] + map_center[0], gt[1] * map_scale[1] + map_center[1]
    gt_bearing.set_xdata([gt[0] * map_scale[0] + map_center[0], gt[0] * map_scale[0] + map_center[0] + 0.5 * map_scale[0] * my_robot.diameter * np.cos(gt[2])])
    gt_bearing.set_ydata([gt[1] * map_scale[1] + map_center[1], gt[1] * map_scale[1] + map_center[1] + 0.5 * map_scale[1] * my_robot.diameter * np.sin(gt[2])])

    od_icon.center = od[0]* map_scale[0]  + map_center[0], od[1] * map_scale[1]  + map_center[1]
    od_bearing.set_xdata([od[0] * map_scale[0] + map_center[0], od[0] * map_scale[0] + map_center[0] + 0.5 * my_robot.diameter * map_scale[0] * np.cos(od[2])])
    od_bearing.set_ydata([od[1] * map_scale[1] + map_center[1], od[1] * map_scale[1] + map_center[1] + 0.5 * my_robot.diameter * map_scale[1] * np.sin(od[2])])

    lidar_values = my_robot.getLidarMeaurement()
    updatedLidarDraw(lidarPoints, gt, lidar_values, my_robot.getLidarResolution(), map_center, map_scale)

    v = my_robot.getLinearVelocity()
    w = my_robot.getAngularVelocity()
    s_text = "time = " + "{:.2f}".format(my_robot.getSimulationTime()) + "seg   u_t=(" + "{:.2f}".format(v)  + " ; " + "{:.2f}".format(w) + ") Collision = " + str(my_robot.getBumperSensorValue()) 
    status_text.set_text(s_text)

    plt.draw()
    file = "/content/plots/_" + str(i).zfill(3)
    plt.savefig(file)
    plt.pause(my_robot.dt)

    # As plot function takes time it is not needed to sleep main thread
    # if it is not the case consider to sleep main thread.
    time.sleep(my_robot.dt)

my_robot.stopSimulation()

In [ ]:
my_robot.stopSimulation()

finishing simulation thread...
Simulation Thread Stopped


In [ ]:
from skimage.draw import line
import numpy as np

In [ ]:
img = np.zeros((10, 10))
rr, cc = line(1, 1, 7, 9)
img[rr, cc] = 1
img
img_aux = 1 - 1/(1+np.exp(img))
img_aux[1,1]

In [ ]:
filtro_de_particulas.particles[2].particle_pos()

array([ 2.46552321,  0.3888285 , -0.29216258])

In [ ]:
%matplotlib notebook

In [ ]:
!zip -r /content/plots.zip /content/plots
files.download("/content/plots.zip")


In [ ]:
!rm -r /content/plots
!rm /content/plots.zip
!mkdir plots

rm: cannot remove '/content/plots.zip': No such file or directory
